# OBSOLETE. CHECK src/.py version

In [ ]:
import boto3
import time
import os
from multiprocessing import Pool, cpu_count
from typing import List

- C:\Users\codenamewei\Documents\xboxgamebar\Captures\greyanatomy2.mp4 32 MB
    - native multipart: Time consumed in seconds: 104.95232319831848s
    - multipart with multiprocessing: 

In [ ]:
filename = r"C:\Users\codenamewei\Documents\xboxgamebar\Captures\insanefood.mp4"
bucketname = "hello-world-abc"
objectname = os.path.basename(filename)

In [ ]:
import json

configfilepath = r"C:\Users\codenamewei\Downloads\temp\awscredential.json"
with open(configfilepath, 'r') as openfile:
 
    # Reading from json file
    config = json.load(openfile)
    
s3client = boto3.client('s3', aws_access_key_id=config["aws_access_key_id"], aws_secret_access_key= config["aws_secret_access_key"])

In [ ]:
import os
import sys
import threading

class ProgressPercentage(object):

    def __init__(self, filename):
        self._filename = filename
        self._size = float(os.path.getsize(filename))
        self._seen_so_far = 0
        self._lock = threading.Lock()

    def __call__(self, bytes_amount):
        # To simplify, assume this is hooked up to a single filename
        with self._lock:
            self._seen_so_far += bytes_amount
            percentage = (self._seen_so_far / self._size) * 100
            print("\r%s  %s / %s  (%.2f%%)" % (
                    self._filename, self._seen_so_far, self._size,
                    percentage), end = "")
     
            sys.stdout.flush()

### https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_multipart_upload
<details>
```
response = client.create_multipart_upload(
    ACL='private'|'public-read'|'public-read-write'|'authenticated-read'|'aws-exec-read'|'bucket-owner-read'|'bucket-owner-full-control',
    Bucket='string',
    CacheControl='string',
    ContentDisposition='string',
    ContentEncoding='string',
    ContentLanguage='string',
    ContentType='string',
    Expires=datetime(2015, 1, 1),
    GrantFullControl='string',
    GrantRead='string',
    GrantReadACP='string',
    GrantWriteACP='string',
    Key='string',
    Metadata={
        'string': 'string'
    },
    ServerSideEncryption='AES256'|'aws:kms',
    StorageClass='STANDARD'|'REDUCED_REDUNDANCY'|'STANDARD_IA'|'ONEZONE_IA'|'INTELLIGENT_TIERING'|'GLACIER'|'DEEP_ARCHIVE'|'OUTPOSTS'|'GLACIER_IR',
    WebsiteRedirectLocation='string',
    SSECustomerAlgorithm='string',
    SSECustomerKey='string',
    SSEKMSKeyId='string',
    SSEKMSEncryptionContext='string',
    BucketKeyEnabled=True|False,
    RequestPayer='requester',
    Tagging='string',
    ObjectLockMode='GOVERNANCE'|'COMPLIANCE',
    ObjectLockRetainUntilDate=datetime(2015, 1, 1),
    ObjectLockLegalHoldStatus='ON'|'OFF',
    ExpectedBucketOwner='string',
    ChecksumAlgorithm='CRC32'|'CRC32C'|'SHA1'|'SHA256'
)
```
<\details>

In [ ]:
def worker_upload_file(param : dict) -> dict:
    """
    (KEY) bucketname <> value : ??
    (KEY) objectname <> value : str
    (KEY) uploadid <> value : int
    (KEY) partnumber <> value : int
    (KEY) contents <> value : bytes
    """
    #https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#multipartuploadpart
    uploadPart = s3resource.MultipartUploadPart(
        bucket_name = param["bucketname"], object_key = param["objectname"], multipart_upload_id = param["uploadid"], part_number = param["partnumber"])

    uploadPartResponse = uploadPart.upload(
        Body=piece,
    )

    #progress(sys.getsizeof(piece))
    print(f"Complete part {part_number}")
    part_number = part_number + 1
    
    return {'PartNumber': part_number, 'ETag': uploadPartResponse['ETag']}

In [ ]:
basename = os.path.basename(filename)

#https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_multipart_upload
multipart_upload = s3client.create_multipart_upload(
    ACL='private',
    Bucket=bucketname,
    ContentType='video/mp4',
    Key=basename,
)

part_number = 1
parts = []

#progress = ProgressPercentage(filename)

s3resource = boto3.resource('s3', aws_access_key_id=config["aws_access_key_id"], aws_secret_access_key= config["aws_secret_access_key"])

sizeperchunk = 10000000# roughly 10 mb parts
start = time.time()

fileidcounter = 1
filechunks : list = []

filechunkbase : dict = {"bucketname": bucketname, "objectname": basename, "uploadid": multipart_upload['UploadId']}

with open(filename, 'rb') as f:
    while True:
        piece = f.read(sizeperchunk) 
        if piece == b'':
            break
        else:
            
            thisfilechunk = filechunkbase
            thisfilechunk["fileid"] = fileidcounter
            thisfilechunk["contents"] = piece
            
            filechunks.append(thisfilechunk)
            fileidcounter += 1

parts = []

#Using as much of workers as per the number of chunks
WORKERS = filechunks if (cpu_count() > filechunks) else cpu_count() 
print(f"Using {WORKERS} multiprocessing workers\n")


with Pool(WORKERS) as p:

    for result in p.imap(worker_upload_file, filechunks):
        parts.append(result)

### To signal completion

In [ ]:
completeResult = s3client.complete_multipart_upload(
    Bucket=bucketname,
    Key=basename,
    MultipartUpload={
        'Parts': parts
    },
    UploadId=multipart_upload['UploadId'],
)

end = time.time()

print(f"Time consumed in seconds: {end - start}s")